In [4]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from xgboost.callback import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.utils import resample
import pandas as pd
from collections import Counter


random_state = 42
np.random.seed(random_state)

In [6]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv')
#ctr_17 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_17.csv') 
#ctr_18 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_18.csv')
#ctr_19 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_19.csv') 
#ctr_20 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_20.csv')    
data = pd.concat(([ctr_15,ctr_16]))
#data = ctr_15
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print("Tamaño del conjunto ", data.shape)
print("Tamaño del conjunto ", ctr_test.shape)


Tamaño del conjunto  (2331937, 52)
Tamaño del conjunto  (1139639, 51)


In [10]:
import pandas as pd
import numpy as np
from collections import Counter

# Supongamos que tienes un DataFrame 'data' con la columna 'Label'
# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['Label'])  # Características (sin la columna objetivo)
y = data['Label']  # Variable objetivo (la columna de la etiqueta)

# Concatenar X e y para trabajar con el DataFrame completo
data_combined = pd.concat([X, y], axis=1)

# Separar las clases
df_class_0 = data_combined[data_combined['Label'] == 0]
df_class_1 = data_combined[data_combined['Label'] == 1]

# Contar el número de muestras de la clase 1
num_class_1 = len(df_class_1)

# Contar el número de muestras de la clase 0
num_class_0 = len(df_class_0)

# Calcular cuántas instancias nuevas necesitamos
n_needed = num_class_0 - num_class_1

# Generar nuevas instancias sintéticas
new_instances = []
for _ in range(n_needed):
    # Elegir aleatoriamente dos instancias de la clase 1
    instance_1 = df_class_1.sample(n=1, replace=True).iloc[0]
    instance_2 = df_class_1.sample(n=1, replace=True).iloc[0]
    
    # Crear una nueva instancia
    new_instance = instance_1.copy()
    
    # Promediar solo las características numéricas
    for col in X.columns:
        if pd.api.types.is_numeric_dtype(instance_1[col]):
            # Asegurarse de que los valores son numéricos
            num1 = pd.to_numeric(instance_1[col], errors='coerce')
            num2 = pd.to_numeric(instance_2[col], errors='coerce')
            new_instance[col] = (num1 + num2) / 2  # Promediar las características numéricas
        else:
            # Para columnas no numéricas, tomar un valor aleatorio de las instancias seleccionadas
            new_instance[col] = np.random.choice([instance_1[col], instance_2[col]])

    # Añadir la nueva instancia a la lista
    new_instances.append(new_instance)

# Convertir la lista de nuevas instancias a un DataFrame
df_new_instances = pd.DataFrame(new_instances, columns=data.columns)

# Combinar las instancias sintéticas con las originales
df_balanced = pd.concat([data_combined, df_new_instances], ignore_index=True)

# Mezclar las filas (opcional)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Ver la nueva distribución de clases
print("Distribución de clases después del oversampling sintético:", Counter(df_balanced['Label']))

/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/pandas/core/dtypes/common.py:1645: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/pandas/core/dtypes/common.py:1645: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/pandas/core/dtypes/common.py:1645: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/pandas/core/dtypes/common.py:1645: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated